In [25]:
import sys

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials,SpotifyOAuth
# import spotipy.oauth2 as oauth2

import json
from pprint import pprint

from configNI import CLIENT_ID, CLIENT_SECRET

In [28]:
#authenticating with client id & client secret credentials

credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)


token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

pprint(spotify)